In [12]:
import numpy as np
from packages.Utility.CNN.Callbacks import *
from packages.Utility.DatasetManager import DatasetReader
from packages.Utility.Gabor import GaborFilterBanks, GaborParams
from packages.Model.LeNet import LeNet

In [13]:
fold_index = 1
dataset_dir = 'dataset/bin/DG.32.A.50K'
trained_dir = 'trained/DG.32.A.50K LN1.1'

# Memuat Dataset

In [14]:
reader = DatasetReader(dataset_dir)

X_train, Y_train_onehot, Y_train_single = reader.getTrainData(fold_index)
X_test, Y_test_onehot, Y_test_single = reader.getTestData(fold_index)

# Inisiasi Callbacks

In [15]:
val_model = ValidateModel(X_train, Y_train_onehot)
save_model = SaveModel(val_model, base_dir = trained_dir, fold = fold_index + 1, min_acc = 0.1)
stop_train = StopTraining(val_model, max_passes = -1)
history_save = HistorySaver(val_model, base_dir = trained_dir, fold = fold_index + 1)

# Parameter Gabor

In [16]:
class RotatedGaborParams(GaborParams):

    def __init__(self, lambd, sigma, psi, gamma):
        self.lambd = lambd
        self.sigma = sigma
        self.psi = psi,
        self.gamma = gamma

    def getParams(self, shape, i, j):
        multiplier = 360 / shape[3]
        degree = i * multiplier
        theta = degree * np.pi / 180
        
        return self.lambd, theta, self.psi, self.sigma, self.gamma

# Mendefinisikan Kernel Gabor

In [17]:
kernel1 = GaborFilterBanks(RotatedGaborParams(psi = 1, gamma = 0.5, lambd = 5, sigma = 3))
kernel2 = GaborFilterBanks(RotatedGaborParams(psi = 1, gamma = 0.75, lambd = 3, sigma = 5))

# Mendefinisikan Model CNN
Mendefinisikan arsitektur CNN

In [18]:
ars = LeNet(kernels = {
    1: kernel1,
    2: kernel2
}, loss = 'categorical_crossentropy')

model = ars.model
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
c1 (Conv2D)                  (None, 28, 28, 6)         156       
_________________________________________________________________
mp1 (MaxPooling2D)           (None, 14, 14, 6)         0         
_________________________________________________________________
c2 (Conv2D)                  (None, 10, 10, 16)        2416      
_________________________________________________________________
mp2 (MaxPooling2D)           (None, 5, 5, 16)          0         
_________________________________________________________________
flatten (Flatten)            (None, 400)               0         
_________________________________________________________________
d1 (Dense)                   (None, 120)               48120     
_________________________________________________________________
d2 (Dense)                   (None, 84)                10164     
__________

In [ ]:
ars.save(f'{trained_dir}/Fold {fold_index + 1}/init.h5')

In [19]:
model.load_weights(f'{trained_dir}/Fold {fold_index + 1}/acc 0.9844 - epoch 383.h5')

# Melakukan Pelatihan
Melakukan pelatihan pada CNN

In [ ]:
ars.compile()
history = model.fit(X_train, Y_train_onehot, epochs = 500, verbose = 2, batch_size = 128, callbacks=[val_model, save_model, history_save, stop_train], initial_epoch=384)